## TPOT (Tree-based Pipeline Optimization Tool) demos

In [1]:
!pip3 install tpot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 6.8 MB/s 
     |████████████████████████████████| 139 kB 63.3 MB/s 
     |████████████████████████████████| 255.9 MB 58 kB/s 
  Created wheel for stopit: filename=stopit-1.1.2-py3-none-any.whl size=11956 sha256=fc47a06cfb1ba30b298bfd8bc8aeac685d75843608586b50b7394d65fc9b02dd
  Stored in directory: /root/.cache/pip/wheels/e2/d2/79/eaf81edb391e27c87f51b8ef901ecc85a5363dc96b8b8d71e3
Successfully built stopit
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


### Classification

In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

x = load_iris().data
y = load_iris().target
labels = load_iris().target_names

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [3]:
from tpot import TPOTClassifier

clf = TPOTClassifier(population_size=10, generations=10, verbosity=2)
clf.fit(x_train, y_train)

Optimization Progress:   0%|          | 0/110 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.9583333333333334

Generation 2 - Current best internal CV score: 0.9666666666666668

Generation 3 - Current best internal CV score: 0.9666666666666668

Generation 4 - Current best internal CV score: 0.9666666666666668

Generation 5 - Current best internal CV score: 0.9666666666666668

Generation 6 - Current best internal CV score: 0.9666666666666668

Generation 7 - Current best internal CV score: 0.9666666666666668

Generation 8 - Current best internal CV score: 0.9666666666666668

Generation 9 - Current best internal CV score: 0.9666666666666668

Generation 10 - Current best internal CV score: 0.9833333333333332

Best pipeline: MultinomialNB(CombineDFs(MLPClassifier(input_matrix, alpha=0.0001, learning_rate_init=0.001), input_matrix), alpha=1.0, fit_prior=False)


TPOTClassifier(generations=10, population_size=10, verbosity=2)

In [4]:
clf.fitted_pipeline_

Pipeline(steps=[('featureunion',
                 FeatureUnion(transformer_list=[('stackingestimator',
                                                 StackingEstimator(estimator=MLPClassifier())),
                                                ('functiontransformer',
                                                 FunctionTransformer(func=<function copy at 0x7f0c4a269cb0>))])),
                ('multinomialnb', MultinomialNB(fit_prior=False))])

In [5]:
clf.export('./tpot_pipeline.py')

In [6]:
with open('./tpot_pipeline.py') as f:
  for line in f:
    print(line, end='')

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline, make_union
from tpot.builtins import StackingEstimator
from sklearn.preprocessing import FunctionTransformer
from copy import copy

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=None)

# Average CV score on the training set was: 0.9833333333333332
exported_pipeline = make_pipeline(
    make_union(
        StackingEstimator(estimator=MLPClassifier(alpha=0.0001, learning_rate_init=0.001)),
        FunctionTransformer(copy)
    ),
    MultinomialNB(alpha=1.0

In [7]:
print('Training accuracy:', clf.score(x_train, y_train).round(3))
print('Test accuracy:', clf.score(x_test, y_test).round(3))

Training accuracy: 0.975
Test accuracy: 1.0


In [8]:
predicted = clf.predict(x_test)

In [9]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predicted, target_names=labels))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      1.00      1.00         9
   virginica       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30



### Regression

In [10]:
import pandas as pd

df = pd.read_csv('https://github.com/PacktPublishing/Automated-Machine-Learning-with-AutoKeras/raw/main/boston.csv')
df

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,6.48,22.0


In [11]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=42)

x_train = train.drop(['MEDV'], axis=1).to_numpy()
y_train = train.MEDV.to_numpy()
x_test = test.drop(['MEDV'], axis=1).to_numpy()
y_test = test.MEDV.to_numpy()

In [12]:
from tpot import TPOTRegressor

reg = TPOTRegressor(population_size=20, generations=20, verbosity=2)
reg.fit(x_train, y_train)

Optimization Progress:   0%|          | 0/420 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: -13.561690144518797

Generation 2 - Current best internal CV score: -12.084627013497919

Generation 3 - Current best internal CV score: -12.084627013497919

Generation 4 - Current best internal CV score: -12.084627013497919

Generation 5 - Current best internal CV score: -11.949633901965852

Generation 6 - Current best internal CV score: -11.949633901965852

Generation 7 - Current best internal CV score: -11.949633901965852

Generation 8 - Current best internal CV score: -11.949633901965852

Generation 9 - Current best internal CV score: -11.949633901965852

Generation 10 - Current best internal CV score: -11.949633901965852

Generation 11 - Current best internal CV score: -11.949633901965852

Generation 12 - Current best internal CV score: -11.779648460799674

Generation 13 - Current best internal CV score: -11.779648460799674

Generation 14 - Current best internal CV score: -11.779648460799674

Generation 15 - Current best internal CV s

TPOTRegressor(generations=20, population_size=20, verbosity=2)

In [13]:
reg.export('./tpot_pipeline.py')

In [14]:
with open('./tpot_pipeline.py') as f:
  for line in f:
    print(line, end='')

import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from sklearn.svm import LinearSVR
from tpot.builtins import StackingEstimator

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
            train_test_split(features, tpot_data['target'], random_state=None)

# Average CV score on the training set was: -9.839246793141
exported_pipeline = make_pipeline(
    PolynomialFeatures(degree=2, include_bias=False, interaction_only=False),
    StackingEstimator(estimator=GradientBoostingRegressor(alpha=0.9, learning_rate=0.1, loss="huber", max_depth=2, m

In [15]:
predicted = reg.predict(x_test)

In [16]:
from sklearn.metrics import r2_score, mean_absolute_error

print('Prection R2:', r2_score(y_test, predicted).round(3))
print('Prection MAE:', mean_absolute_error(y_test, predicted).round(3))

Prection R2: 0.86
Prection MAE: 2.025
